## load relation description

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json
relation_description = {}

In [0]:
with open('/content/gdrive/My Drive/master_thesis/relation_descriptions.json', 'r') as f:
  relation_description = json.load(f)


In [3]:
len(relation_description)

120

## download ppdb

In [6]:
!wget http://nlpgrid.seas.upenn.edu/PPDB/eng/ppdb-2.0-tldr.gz

--2019-06-15 19:07:33--  http://nlpgrid.seas.upenn.edu/PPDB/eng/ppdb-2.0-tldr.gz
Resolving nlpgrid.seas.upenn.edu (nlpgrid.seas.upenn.edu)... 158.130.57.54
Connecting to nlpgrid.seas.upenn.edu (nlpgrid.seas.upenn.edu)|158.130.57.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1782973884 (1.7G) [application/x-gzip]
Saving to: ‘ppdb-2.0-tldr.gz’

ppdb-2.0-tldr.gz    100%[===================>]   1.66G  91.9MB/s    in 18s     

2019-06-15 19:07:51 (95.3 MB/s) - ‘ppdb-2.0-tldr.gz’ saved [1782973884/1782973884]



In [0]:
!gunzip ppdb-2.0-tldr.gz

In [8]:
!head ppdb-2.0-tldr

[NN] ||| transplant ||| transplantation ||| PPDB2.0Score=5.24981 PPDB1.0Score=3.295900 -logp(LHS|e1)=0.18597 -logp(LHS|e2)=0.14031 -logp(e1|LHS)=11.83583 -logp(e1|e2)=1.80507 -logp(e1|e2,LHS)=1.46728 -logp(e2|LHS)=11.47593 -logp(e2|e1)=1.49083 -logp(e2|e1,LHS)=1.10738 AGigaSim=0.63439 Abstract=0 Adjacent=0 CharCountDiff=5 CharLogCR=0.40547 ContainsX=0 Equivalence=0.371472 Exclusion=0.000344 GlueRule=0 GoogleNgramSim=0.03067 Identity=0 Independent=0.078161 Lex(e1|e2)=9.64663 Lex(e2|e1)=59.48919 Lexical=1 LogCount=4.67283 MVLSASim=NA Monotonic=1 OtherRelated=0.372735 PhrasePenalty=1 RarityPenalty=0 ForwardEntailment=0.177287 SourceTerminalsButNoTarget=0 SourceWords=1 TargetComplexity=0.98821 TargetFormality=0.98464 TargetTerminalsButNoSource=0 TargetWords=1 UnalignedSource=0 UnalignedTarget=0 WordCountDiff=0 WordLenDiff=5.00000 WordLogCR=0 ||| 0-0 ||| OtherRelated
[JJ] ||| <www.un.org/depts/dgacm/docs/crp/aconf212crp1/russian.pdf> ||| <www.un.org/depts/dgacm/docs/crp/aconf212crp1/arabic.

### load ppdb to list

In [0]:
ppdb = []

chosen_entailment = ['Equivalence', 'ReverseEntailment', 'ForwardEntailment']
other_entailment = []

with open('ppdb-2.0-tldr', 'r') as f:
  for l in f:
    line = l.split("|||")
    entailment = line[-1].strip()
    if entailment not in chosen_entailment:
      if entailment not in other_entailment:
        other_entailment.append(entailment)
      continue
    lhs = line[0].strip()
    phase = line[1].strip()
    paraphase = line[2].strip()
    try:
      ppdb2_score = eval(line[3].strip().split(" ")[0].split("=")[1])
    except:
      continue
    state = (lhs, phase, paraphase, ppdb2_score, entailment)
    ppdb.append(state)

In [5]:
other_entailment

['OtherRelated', 'Independent', 'Exclusion']

In [6]:
len(ppdb)

4977076

In [7]:
ppdb[100200]

('[NNP]', 'unlimited', 'unrestricted', 2.75464, 'Equivalence')

### transfer ppdb list to dict

In [0]:
ppdb_dict = {}

for tp in ppdb:
  p1 = tp[1]
  p2 = tp[2]
  others = (tp[0], tp[3], tp[4])
  if p1 in ppdb_dict.keys():
    ppdb_dict[p1][p2] = others
  else:
    ppdb_dict[p1]={}
    ppdb_dict[p1][p2] = others

In [9]:
len(ppdb_dict)

868649

In [10]:
ppdb_dict['wife']

{'ex-wife': ('[NN]', 2.80051, 'ReverseEntailment'),
 'spouse': ('[NN]', 2.86243, 'ForwardEntailment'),
 'woman': ('[RB]', 2.81832, 'ForwardEntailment'),
 'women': ('[NN]', 3.1255, 'ForwardEntailment')}

In [11]:
ppdb_dict['woman']

{'female': ('[UH]', 2.77627, 'ReverseEntailment'),
 'girl': ('[NN]', 2.88167, 'ReverseEntailment'),
 'lady': ('[NN]', 3.02029, 'ReverseEntailment'),
 'many women': ('[NP]', 3.70595, 'ReverseEntailment'),
 'men women': ('[TOP]', 3.76014, 'ReverseEntailment'),
 'wife': ('[RB]', 2.80553, 'ReverseEntailment'),
 "women 's issues": ('[NP]', 3.75149, 'ReverseEntailment'),
 "women 's lives": ('[NP]', 3.59837, 'ReverseEntailment'),
 'women and men': ('[NP]', 4.17704, 'ReverseEntailment'),
 'women members': ('[NP]', 3.82372, 'ReverseEntailment'),
 'working women': ('[NP]', 4.13175, 'ReverseEntailment'),
 'young women': ('[NP]', 3.82458, 'ReverseEntailment')}

### sort element in dict according to ppdb2 score

In [0]:
sort_ppdb = {}

for word in ppdb_dict:
  syn_list = [(k,ppdb_dict[word][k]) for k in ppdb_dict[word]]
  sorted_syn_list = sorted(syn_list, key=lambda x: x[1][1], reverse=True)
  sorted_list = [a[0] for a in sorted_syn_list]
  sort_ppdb[word] = sorted_list

In [13]:
ppdb_dict['replaced']

{'replace': ('[NNP]', 3.198, 'Equivalence'),
 'replace the word': ('[VP]', 3.60615, 'ReverseEntailment'),
 'replace the words': ('[VP]', 3.70402, 'ReverseEntailment'),
 'substituted': ('[VBN]', 3.49128, 'Equivalence'),
 'superseded': ('[VBD]', 3.4851, 'Equivalence'),
 'was substituted': ('[VP]', 3.69468, 'ReverseEntailment')}

In [14]:
sort_ppdb['replaced']

['replace the words',
 'was substituted',
 'replace the word',
 'substituted',
 'superseded',
 'replace']

## Fill to the descriptions

### Categorizing and Tagging Words

In [15]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
relation_description['chairperson']

['OBJECT_ENTITY led SUBJECT_ENTITY\n']

In [17]:
des_token = word_tokenize('OBJECT_ENTITY led SUBJECT_ENTITY\n')
nltk.pos_tag(des_token)

[('OBJECT_ENTITY', 'NNP'), ('led', 'VBD'), ('SUBJECT_ENTITY', 'NNP')]

In [18]:
# check POS
for relation in relation_description:
  for desc in relation_description[relation]:
    des_token = word_tokenize(desc)
    print(nltk.pos_tag(des_token))

[('OBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('airport', 'NN'), ('at', 'IN'), ('which', 'WDT'), ('you', 'PRP'), ('can', 'MD'), ('find', 'VB'), ('SUBJECT_ENTITY', 'JJ')]
[('The', 'DT'), ('airport', 'NN'), ('at', 'IN'), ('which', 'WDT'), ('SUBJECT_ENTITY', 'NNP'), ('operates', 'VBZ'), ('out', 'IN'), ('of', 'IN'), ('is', 'VBZ'), ('OBJECT_ENTITY', 'NNP')]
[('OBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('airport', 'NN'), ('SUBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('associated', 'VBN'), ('with', 'IN')]
[('OBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('airport', 'NN'), ('which', 'WDT'), ('SUBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('part', 'NN'), ('of', 'IN')]
[('The', 'DT'), ('name', 'NN'), ('of', 'IN'), ('the', 'DT'), ('airport', 'NN'), ('associated', 'VBN'), ('with', 'IN'), ('SUBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('OBJECT_ENTITY', 'NNP')]
[('OBJECT_ENTITY', 'NNP'), ('is', 'VBZ'), ('the', 'DT'), ('name', 'NN'), ('of', 'IN'), ('the', 'DT'), ('airport

### helper functions for extending descriptions

In [0]:
import numpy as np

def levenshtein(seq1, seq2):  
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])


def comp(a, key_word_set):
  closest = float('inf')
  for ele in key_word_set:
    dist = levenshtein(ele, a)
    if dist < closest:
      closest = dist
  return closest


def collect_descriptions(relation_sk, desc):
  
  new_description_pool = []
  
  for des in desc:
    
    des_token = word_tokenize(des)
    des_tag = nltk.pos_tag(des_token)
    
    nn = [wd for (wd, tag) in des_tag if tag in ['NN', 'NNS', 'VBN', 'VBD', 'VBZ', 'VBG', 'VBP', 'JJ'] and wd not in ['is', 'are', 'was', 'were']]
    snn = sorted(nn, key=lambda x: comp(x, relation_sk)) # sorted list of NN word in description, order indicates the importance
    for sk in relation_sk:
      snn.insert(0, sk)
    
    current_key_pool = []
    for key_word in snn:
      if key_word in sort_ppdb:
        kwd_pair = (key_word, sort_ppdb[key_word])
        current_key_pool.append(kwd_pair)
    
    current_des_pool = []
    for (kw, sort_nn_list) in current_key_pool:
      for nn in sort_nn_list:
        new_des = des.replace(kw, nn)
        current_des_pool.append(new_des)
    
    new_description_pool.append(current_des_pool)
    
  result_desc = desc
  visited_path = []
  for extends_from_one_des in new_description_pool:
    for one_extend in extends_from_one_des:
      if one_extend not in visited_path and one_extend not in result_desc:
        visited_path.append(one_extend)
        result_desc.append(one_extend)
      else:
        continue
      if len(result_desc) >= 50:
        return result_desc
  return result_desc

### First extend

In [0]:
# define the not important part of relation
description_debris = ['of', 'at', 'into', 'by', 'on', 'or', 'after', 'for']

# target full descriptions
full_relation_description = {}

# for each relation, generating 50 descriptions
for relation in relation_description:
  desc = relation_description[relation].copy()
  
  # collecting the import keywords of each relation
  relation_list = relation.split()
  relation_sk = {relation} # a set of important keyword of this relation
  
  for de_wd in relation_list:
    if de_wd not in description_debris:
      relation_sk.add(de_wd)
    else:
      continue
  
  final_desc = collect_descriptions(relation_sk, desc)
  full_relation_description[relation] = final_desc
    

In [21]:
less_than_5 = 0
less_than_10 = 0
for relation in relation_description:
  desc = relation_description[relation]
  desc_num = len(desc)
  print(relation, desc_num)
  if desc_num <= 10:
    less_than_10 += 1
    if desc_num <= 5:
      less_than_5 += 1

airline hub 9
architect 29
architectural style 3
author 28
award received 5
based on 2
brother 2
canonization status 1
cast member 10
cause of death 24
chairperson 1
characters 1
child 2
chromosome 12
collection 1
conferred by 1
conflict 25
connecting line 11
constellation 23
continent 18
convicted of 3
country 26
country of citizenship 18
country of origin 6
creator 7
crosses 4
date of birth 9
date of death 21
date of official opening 14
designer 10
developer 9
director 26
discoverer or inventor 15
dissolved or abolished 9
distributor 20
drafted by 7
editor 4
educated at 8
employer 1
end time 1
father 14
film editor 18
founder 10
found in taxon 3
from fictional universe 12
head of government 4
headquarters location 6
home venue 29
illustrator 4
inception 27
industry 1
instrumentation 15
instrument 25
IUCN conservation status 8
language of work or name 9
languages spoken or written 12
league 8
licensed to broadcast to 3
license 1
located in the administrative territorial entity 4
locat

In [22]:
less_than_5, less_than_10

(49, 77)

In [23]:
for relation in full_relation_description:
  desc = full_relation_description[relation]
  desc_num = len(desc)
  if desc_num < 15:
    print(relation, desc_num)

based on 11
chairperson 3
crosses 13
industry 2
license 6
residence 8
spouse 13
taxon rank 14


### Second extend

In [0]:
# mannually add those relations with less than 15 description after expend (造句)
mannual_relation_description = relation_description.copy()

# based on
desc = mannual_relation_description['based on'] # copy by reference
desc.append('The SUBJECT_ENTITY is based on OBJECT_ENTITY\n')

# chairperson, The original one is wrong
mannual_relation_description['chairperson'] = ['SUBJECT_ENTITY led OBJECT_ENTITY\n'] # original: 'OBJECT_ENTITY led SUBJECT_ENTITY\n'
desc = mannual_relation_description['chairperson']
desc.append('The SUBJECT_ENTITY is the chairperson of OBJECT_ENTITY\n')
desc.append('The SUBJECT_ENTITY is the leader of OBJECT_ENTITY\n')
desc.append('The SUBJECT_ENTITY is leading OBJECT_ENTITY\n')

# characters
desc = mannual_relation_description['characters']
desc.append('The SUBJECT_ENTITY has OBJECT_ENTITY property\n')

# conferred by
desc = mannual_relation_description['conferred by']
desc.append('SUBJECT_ENTITY is conferred by OBJECT_ENTITY\n')
desc.append('OBJECT_ENTITY grant SUBJECT_ENTITY\n')

# crosses
desc = mannual_relation_description['crosses']
desc.append('SUBJECT_ENTITY travel across the river OBJECT_ENTITY\n')
desc.append('SUBJECT_ENTITY pass over OBJECT_ENTITY\n')

# industry
desc = mannual_relation_description['industry']
desc.append('SUBJECT_ENTITY is associated with OBJECT_ENTITY economic field\n')
desc.append('SUBJECT_ENTITY is associated with OBJECT_ENTITY branch\n')

# license
desc = mannual_relation_description['license']
desc.append('OBJECT_ENTITY is the certificate of SUBJECT_ENTITY\n')
desc.append('OBJECT_ENTITY is the permit of SUBJECT_ENTITY\n')

# occupant
desc = mannual_relation_description['occupant']
desc.append('OBJECT_ENTITY is the occupant of SUBJECT_ENTITY\n')
desc.append('OBJECT_ENTITY is the occupier of SUBJECT_ENTITY\n')
desc.append('OBJECT_ENTITY is the occupied by SUBJECT_ENTITY\n')
desc.append('SUBJECT_ENTITY occupy OBJECT_ENTITY\n')

# residence
desc = mannual_relation_description['residence']
desc.append('SUBJECT_ENTITY is resident in OBJECT_ENTITY\n')
desc.append('SUBJECT_ENTITY is located in OBJECT_ENTITY\n')

# spouse
desc = mannual_relation_description['spouse']
desc.append('SUBJECT_ENTITY is the consort of OBJECT_ENTITY\n')
desc.append('SUBJECT_ENTITY is the husband of OBJECT_ENTITY\n')
desc.append('SUBJECT_ENTITY is the wife of OBJECT_ENTITY\n')

#taxon rank
desc = mannual_relation_description['taxon rank']
desc.append('SUBJECT_ENTITY is the taxonomic rank of OBJECT_ENTITY\n')


In [0]:
manual_full_relation_description = {}

for relation in mannual_relation_description:
  desc = mannual_relation_description[relation].copy()
  
  # collecting the import keywords of each relation
  relation_list = relation.split()
  relation_sk = {relation} # a set of important keyword of this relation
  
  for de_wd in relation_list:
    if de_wd not in description_debris:
      relation_sk.add(de_wd)
    else:
      continue
  
  final_desc = collect_descriptions(relation_sk, desc)
  manual_full_relation_description[relation] = final_desc
    

In [26]:
for relation in manual_full_relation_description:
  desc = manual_full_relation_description[relation]
  desc_num = len(desc)
  print(relation, desc_num)

airline hub 50
architect 50
architectural style 27
author 50
award received 50
based on 50
brother 28
canonization status 22
cast member 50
cause of death 50
chairperson 21
characters 28
child 50
chromosome 50
collection 30
conferred by 29
conflict 50
connecting line 50
constellation 50
continent 50
convicted of 50
country 50
country of citizenship 50
country of origin 50
creator 50
crosses 28
date of birth 50
date of death 50
date of official opening 50
designer 50
developer 50
director 50
discoverer or inventor 50
dissolved or abolished 50
distributor 50
drafted by 50
editor 36
educated at 50
employer 25
end time 35
father 50
film editor 50
founder 50
found in taxon 50
from fictional universe 50
head of government 50
headquarters location 50
home venue 50
illustrator 50
inception 50
industry 16
instrumentation 50
instrument 50
IUCN conservation status 50
language of work or name 50
languages spoken or written 50
league 50
licensed to broadcast to 46
license 22
located in the administ

### export file

In [0]:
with open('extended_relation_descriptions.json', 'w') as f:
  json.dump(manual_full_relation_description,f)